<a href="https://colab.research.google.com/github/AlexandreBourrieau/ML/blob/main/Carnets%20Jupyter/Fraude_CB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Détection de fraudes à la carte bancaire

Cette base de données contient des transactions effectuées par CB en septembre 2013 par des utilisateurs européens. Les données ont été enregistrées sur deux jours, pendant lesquels 492 utilisations frauduleuses sur un total de 284 807 transactions ont été enregistrées.  

Ce ratio est fortement déséquilibré car les cas de fraudes ne représentent que 0.172% des transactions. Un apprentissage supervisé n'est donc pas possible ici.  

Nous allons utiliser un auto-encodeur pour effectuer un apprentissage non supervisé afin de détecter des irrégularités dans les données, indices de fraudes.  

Les données sont disponibles sur le site de [Kaggle](https://www.kaggle.com/mlg-ulb/creditcardfraud)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.metrics import confusion_matrix

## Lecture des données

Le nombre de données est très grand. Pour réduire le temps de calcul, nous n'utiliserons qu'une partie d'entre elles.  

In [ ]:
!wget --no-check-certificate --content-disposition "https://github.com/AlexandreBourrieau/ML/blob/main/Carnets%20Jupyter/Donn%C3%A9es/creditcard.zip?raw=true"
!unzip creditcard.zip -x

In [ ]:
!ls -l

In [ ]:
data = pd.read_csv("creditcard.csv")
data = data.head(30000)
data.shape

Les colonnes qui ne sont pas anonymisées sont :
- Time : Temps écoulé depuis la première transaction
- Amount : Montant de la transaction
- Class : 0 = Achat non frauduleux / 1 = achat frauduleux
  
Les autres données anonymes sont par exemple les coordonnées de l'acheteur, son adresse, ...

In [ ]:
data.head()

Regardons combien nous avons de données frauduleuses :

In [ ]:
data.groupby(['Class']).count()

In [ ]:
100*(94/29906)

## Préparation des données

La colonne Time tout d'abord ignorée

In [ ]:
data = data.drop(['Time'], axis=1)

Ensuite, les montants des transactions sont normalisés en supprimant la valeur moyenne et en faisant en sorte que la variance soit égale à 1. Les autres colonnes sont inchangées.

In [ ]:
# Histogramme des montants avant normalisation

df = data['Amount']
print(df.describe())
df.hist(bins=100)

In [ ]:
# Normalisation des montants

scaler = StandardScaler()
data['Amount'] = scaler.fit_transform(data['Amount'].values.reshape(-1, 1))

In [ ]:
# Histogramme des montants après normalisation

df = data['Amount']
print(df.describe())
df.hist(bins=100)

In [ ]:
# Séparation des données d'entrainement et de test

x_entrainement, x_test = train_test_split(data, test_size=0.2, random_state=0)
x_entrainement = x_entrainement.drop(['Class'], axis=1)
y_test = x_test['Class']
x_test = x_test.drop(['Class'], axis=1)
x_entrainement = x_entrainement.values
x_test = x_test.values
x_entrainement.shape

## Construction du modèle

Ce modèle est un modèle standard d'auto-encodeur à quatre couches :
- Une couche d'entrée à 14 neurones, avec une fonction d'activation de type tanh
- Une première couche cachées de 14/2 = 7 neurones, avec une fonction d'activation de type relu
- Une deuxième couche cachée de 14/2 = 7 neurones, avec une fonction d'activation de type tanh
- Une couche de sortie à 29 neurones (même dimension que les données d'entrées), avec une fonction d'activation de type relu

La structure de notre modèle peut être visualisée comme ci-dessous :  
![picture](https://github.com/AlexandreBourrieau/ML-F1/blob/master/Carnets%20Jupyter/Images/Schema_Non_Supervis%C3%A932.png?raw=true "ReseauNeurone")

In [ ]:
dimension_entrees = x_entrainement.shape[1]
dimension_encodeur = 14

model = Sequential()
model.add(Dense(dimension_encodeur, activation="tanh", input_shape=(dimension_entrees,)))
model.add(Dense(int(dimension_encodeur /2), activation="relu"))
model.add(Dense(int(dimension_encodeur /2), activation='tanh'))
model.add(Dense(dimension_entrees, activation='relu'))

model.summary()

# Entrainement du modèle



In [ ]:
nb_iterations = 50
batch_size = 30

model.compile(optimizer='adam', loss='mean_squared_error', metrics=['acc'])

history = model.fit(x_entrainement, x_entrainement,
                    epochs=nb_iterations,
                    batch_size=batch_size,
                    validation_data=(x_test, x_test),
                    verbose=1)

autoencoder = model

## Performances du modèle


In [ ]:
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Précision')
plt.xlabel('Itérations')
plt.legend(['Entrainement', 'Test'], loc='upper left')
plt.show()

## Structure du modèle et calcul de l'erreur de prédiction
La structure de notre modèle peut être visualisée comme ci-dessous :  
![picture](https://github.com/AlexandreBourrieau/ML-F1/blob/master/Carnets%20Jupyter/Images/Schema_Non_Supervis%C3%A932.png?raw=true "ReseauNeurone")

L'erreur de prédiction est l'erreur quadratique moyenne entre les entrées et les sorties (écart type). Pour chaque couple d'entrée / sortie, on calcule donc la valeur suivante :

 $Erreur =  \sqrt {{{({V_{1{\rm{\_sortie}}}} - {V_{1{\rm{\_entr}}e }})}^2} + {{({V_{2{\rm{\_sortie}}}} - {V_{2{\rm{\_entr}}e }})}^2} + ... + {{(Amoun{t_{{\rm{\_sortie}}}} - Amoun{t_{{\rm{\_entr}}e }})}^2}} $

In [ ]:
x_test

## Prédictions

Les prédictions sont effectuées sur les données de tests. L'erreur quadratique moyenne (mean-squared error - MSE) est calculés entre les tests et les prédictions. Si cette erreur est grande, c'est une potentielle anomalie et donc une fraude potentielle. Bien sûr, cela n'est pas parfait et il y a des faux-positifs (et des faux négatifs) !

In [ ]:
predictions = autoencoder.predict(x_test)
mse = np.mean(np.power(x_test - predictions, 2), axis=1)
error_df = pd.DataFrame({'erreur_reconstruction': mse, 'class': y_test})

In [ ]:
x_test.shape

In [ ]:
predictions
mse

In [ ]:
error_df.head()

## Affichage de l'erreur de reconstruction

L'erreur de reconstruction est affichée pour chaque échantillon. Seuls 6000 échantillons sont affichés, mais les données affichées sont tirées au hasard.

In [ ]:
niveau_detection = 7

groups = error_df.groupby('class')
fig, ax = plt.subplots(figsize=(12, 8))

for name, group in groups:
    ax.plot(group.index, group.erreur_reconstruction, marker='o', ms=2.0, linestyle='',
            label = "Fraude" if name == 1 else "Normal",
            color = "red" if name == 1 else "blue")
ax.hlines(niveau_detection, ax.get_xlim()[0], ax.get_xlim()[1], colors="green", zorder=100, label='Niveau de détection')
ax.legend()
plt.title("Erreur de reconstruction pour différents échantillons")
plt.ylabel("Erreur de reconstruction")
plt.xlabel("Index de l'échantillon")
plt.show();

# Analyse

Comme nous connaissons les transactions frauduleuses, nous pouvons afficher et calculer le nombre de faux positifs. Dans l'idéal, il faudrait que leur nombre soit très faible, mais notre modèle est imparfait... 

In [ ]:
transactions_normales = error_df[error_df['class'] == 0]
transactions_frauduleuses = error_df[error_df['class'] == 1]

print('Transactions normales : %d, Transactions frauduleuse : %d' % (len(transactions_normales), len(transactions_frauduleuses)))

In [ ]:
transactions_frauduleuses

In [ ]:
vraies_transactions_frauduleuses = len(transactions_frauduleuses[transactions_frauduleuses['erreur_reconstruction'] >= niveau_detection])
vraies_transactions_normales = len(transactions_normales[transactions_normales['erreur_reconstruction'] < niveau_detection])

faux_positifs = len(transactions_normales[transactions_normales['erreur_reconstruction'] >= niveau_detection])
faux_negatifs = len(transactions_frauduleuses[transactions_frauduleuses['erreur_reconstruction'] < niveau_detection])

print('Faux positifs: %d, Vraies transactions frauduleuses: %d' % (faux_positifs, vraies_transactions_frauduleuses))
print('Faux négatifs : %d, Vraies transactions normales: %d' % (faux_negatifs, vraies_transactions_normales))

## Matrice de synthèse

La matrice de synthèse ci-dessous permet d'avoir une vue d'ensemble des résultats obtenus.

In [ ]:
labels = ["Normale", "Fraude"]

y_pred = [1 if e > niveau_detection else 0 for e in error_df.erreur_reconstruction.values]
conf_matrix = confusion_matrix(error_df['class'], y_pred)

plt.figure(figsize=(6, 5))
sns.heatmap(conf_matrix, xticklabels=labels, yticklabels=labels, annot=True, fmt="d");
plt.title("Matrice de synthèse")
plt.ylabel('Cas réels')
plt.xlabel('Prédictions')
plt.show()